In [13]:
import os
import json
import time
import faiss
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from contextlib import contextmanager
from typing import List, Tuple, NoReturn, Any, Optional, Union
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import (
    AutoTokenizer,
    TrainingArguments, 
    BertPreTrainedModel,
    BertModel,AdamW,
    get_linear_schedule_with_warmup,
    RobertaConfig, 
    RobertaModel,
    PreTrainedModel)
from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
    Features,
    Value,
    DatasetDict,
)
import torch.nn.functional as F
from retrieval import *
from tqdm import tqdm
import torch
import random
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
import re
def wiki_preprocess(text):
            text = re.sub(r'\n', ' ', text)
            text = re.sub(r"\\n", " ", text)
            text = re.sub(r'\\n\\n', ' ', text)
            text = re.sub(r'\n\n', " ", text)
            text = re.sub(r"\s+", " ", text)
            text = re.sub(r'#', ' ', text)
            return text 

# 난수 고정
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    random.seed(random_seed)
    np.random.seed(random_seed)
    
set_seed(42) # magic number :)

model_checkpoint = "klue/bert-base"
dataset = "../data/train_dataset"
org_dataset = load_from_disk(dataset)
train_datasets = org_dataset['train']


In [11]:
dataset = "../data/added_dataset"
org_dataset = load_from_disk(dataset)

['도자와시가 참석한 회의의 결과는 언제 발표되었나요?',
 '아이제나흐파(派)의 중요인물은 누구인가요?',
 '펠리페는 어떤 나라를 대표하여 바르셀로나 올림픽에 참가하였는가?',
 '영국 해군이 다시 후장포를 사용하기로 한 것은 몇 년도인가?',
 '제파 디 미노는 친구의 아내에게 함께 복수하기를 부탁하며 무엇을 건네었나?',
 '당나라군의 몰락에 결정적인 역할을 한 세력은 누구인가?',
 '일본 역사상 처음으로 패배한 무사가 할복하지 않게 도와준 사람은?',
 '존 릿지는 누구와 정치적인 힘겨루기를 하였나?',
 '쿠 훌린의 아들이 콘늘라임을 알고 쿠 훌린에게 알리려 한 사람은?',
 '호키가와 강 하부와 연결되어있는 강은?',
 '게오르크의 음악에 대한 천재성은 언제부터 사람들에게 보여졌나요?',
 '마르크수 주의를 주장하는 사람들이 자본국가를 어떻게 지칭하는가?',
 '전기배선용에는 무엇에 의한 손상을 막기 위해 온도조정기능이 있나요?',
 '윤치호가 미국을 더이상 정의와 자유의 국가라 여기지 않게 된 것은 어떤 조약 때문인가?',
 '데네볼라가 에너지 생산을 위해 연소시키는 물질은 무엇인가?',
 '신한청년당이 파리강화회의에 특파한 인물은 누구인가?',
 '오영제의 직업은 무엇인가?',
 '선교사들과 함께 잠자리와 음식을 제공받은 인물은 누구인가?',
 '동관이 요나라를 침략하기 위해 동맹을 맺은 나라는?',
 '그림 형제가 처음 엮어낸 책은?',
 '상품화폐의 인플레이션을 제재하지 못하도록 하는 것은?',
 '터널 내의 환기 방식은 횡류식에서 어떤 방식으로 바뀌었나요?',
 '해왕성의 불규칙 위성 중 해왕성과 가장 멀리 떨어진 것은?',
 '고려 고종 때 유일하게 구제약으로 쓰였던 것은 무엇인가요?',
 '현생 양서류의 호흡방식을 이용해 몸 밖으로 내보낼 수 있는 것은?',
 '키루스가 형을 몰래 죽이려 했던 계획을 사전에 고발했던 인물은?',
 '철불상이 많이 제작된 시대는 언제인가?',
 '마블고양이가 다른 고양이들에 비해 더 긴 것은 무엇인가요

In [2]:

valid_dataset = org_dataset['validation']


from transformers import AutoTokenizer
import numpy as np

#tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)

q_seqs = tokenizer(train_datasets['question'], padding="max_length", truncation=True, return_tensors='pt')
p_seqs = tokenizer(train_datasets['context'], padding="max_length", truncation=True, return_tensors='pt')

train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'], 
                        q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'])
# train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], 
#                         q_seqs['input_ids'], q_seqs['attention_mask'])

class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    self.bert = BertModel(config)
    self.init_weights()
      
  def forward(self, input_ids, 
              attention_mask=None, token_type_ids=None): 
  
      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)
      
      pooled_output = outputs[1]

      return pooled_output


# load pre-trained model on cuda (if available)
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
  p_encoder.cuda()
  q_encoder.cuda()


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.bi

In [5]:

args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=11,
    weight_decay=0.01
)


In [6]:
from transformers import AdamW
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
            {"params": [p for n, p in p_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
            {"params": [p for n, p in p_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
            {"params": [p for n, p in q_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
            {"params": [p for n, p in q_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0}
        ]
optimizer = AdamW(
            optimizer_grouped_parameters,
            lr=args.learning_rate,
            eps=args.adam_epsilon
        )

def train(args, dataset, p_model, q_model):
  
  # Dataloader
  train_sampler = RandomSampler(dataset)
  train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=args.per_device_train_batch_size)

  t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)

  # Start training!
  global_step = 0
  
  p_model.zero_grad()
  q_model.zero_grad()
  torch.cuda.empty_cache()
  
  train_iterator = trange(int(args.num_train_epochs), desc="Epoch")

  for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")

    for step, batch in enumerate(epoch_iterator):
      q_encoder.train()
      p_encoder.train()
      
      if torch.cuda.is_available():
        batch = tuple(t.cuda() for t in batch)

      p_inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids': batch[2]
                  }
      
      q_inputs = {'input_ids': batch[3],
                  'attention_mask': batch[4],
                  'token_type_ids': batch[5]}

      p_outputs = p_model(**p_inputs)  # (batch_size, emb_dim)
      q_outputs = q_model(**q_inputs)  # (batch_size, emb_dim)


      # Calculate similarity score & loss
      sim_scores = torch.matmul(q_outputs, torch.transpose(p_outputs, 0, 1))  # (batch_size, emb_dim) x (emb_dim, batch_size) = (batch_size, batch_size)

      # target: position of positive samples = diagonal element 
      targets = torch.arange(0, args.per_device_train_batch_size).long()
      if torch.cuda.is_available():
        targets = targets.to('cuda')

      sim_scores = F.log_softmax(sim_scores, dim=1)

      loss = F.nll_loss(sim_scores, targets)

      loss.backward()
      optimizer.step()
      scheduler.step()
      q_model.zero_grad()
      p_model.zero_grad()
      global_step += 1
      
      torch.cuda.empty_cache()


    
  return p_model, q_model


In [6]:
p_encoder, q_encoder = train(args, train_dataset, p_encoder, q_encoder)

Epoch: 100%|██████████| 11/11 [1:00:29<00:00, 329.97s/it]


In [3]:
from transformers import BertConfig
config = BertConfig.from_json_file('./qe/config.json')
q_encoder = BertEncoder.from_pretrained('./qe/pytorch_model.bin', config=config)
config = BertConfig.from_json_file('./pe/config.json')
p_encoder = BertEncoder.from_pretrained('./pe/pytorch_model.bin', config=config)

In [4]:
if torch.cuda.is_available():
  p_encoder.cuda()
  q_encoder.cuda()


In [5]:
with open("../data/wikipedia_documents.json", "r", encoding="utf-8") as f:
            wiki = json.load(f)

contexts = list(
            dict.fromkeys([v["text"] for v in wiki.values()])
        ) 
query = valid_dataset['question']
ground_truth = valid_dataset['context']
valid_corpus = contexts
# query = train_datasets['question'][:1000]
# ground_truth = train_datasets['context'][:1000]
# valid_corpus = train_datasets['context'][:1000]

def to_cuda(batch):
  return tuple(t.cuda() for t in batch)

In [6]:
for idx in range(len(contexts)):
    contexts[idx] = wiki_preprocess(contexts[idx])

In [7]:
for idx in range(len(ground_truth)):
    ground_truth[idx] = wiki_preprocess(ground_truth[idx])

In [8]:
len(contexts), len(list(set(contexts)))

(56737, 55963)

In [9]:
contexts = list(set(contexts))
valid_corpus = contexts

In [10]:
crt = 0
for i in ground_truth:
    if i in valid_corpus:
        crt+=1
print(crt)

240


In [13]:
with torch.no_grad():
  p_encoder.eval()
  q_encoder.eval()

  q_seqs_val = tokenizer(query, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
  q_emb = q_encoder(**q_seqs_val).to('cpu')  #(num_query, emb_dim)

  p_embs = []
  for p in valid_corpus:
    p = tokenizer(p, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
    p_emb = p_encoder(**p).to('cpu').numpy()
    p_embs.append(p_emb)

p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)

print(p_embs.size(), q_emb.size())

torch.Size([55963, 768]) torch.Size([240, 768])


In [20]:

dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
print(dot_prod_scores.size())

ranks = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()

NameError: name 'q_emb' is not defined

In [21]:
k = 15
context_list = []

for index in range(len(ranks)):
    k_list = []
    for i in range(k):
        k_list.append(valid_corpus[ranks[index][i]])
    context_list.append(k_list)
correct= 0
for index in range(len(context_list)):
    if ground_truth[index] in context_list[index]:
        correct+=1 
print(correct/len(context_list))

0.625


In [11]:
from torch.nn.functional import softmax

In [22]:
q_encoder.save_pretrained("./qe")
p_encoder.save_pretrained("./pe")

In [4]:
from transformers import BertConfig
config = BertConfig.from_json_file('./qe/config.json')
q_encoder = BertEncoder.from_pretrained('./qe/pytorch_model.bin', config=config)

In [5]:
config = BertConfig.from_json_file('./pe/config.json')
p_encoder = BertEncoder.from_pretrained('./pe/pytorch_model.bin', config=config)

In [13]:
if torch.cuda.is_available():
  p_encoder.cuda()
  q_encoder.cuda()


In [12]:
tokenize_fn = tokenizer.tokenize

In [24]:
len(contexts)

55963

In [13]:
with torch.no_grad():
  p_encoder.eval()
  tokenized_corpus = []
  p_embs = []
  for p in tqdm(contexts):
    p_t = tokenizer(p, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
    p_emb = p_encoder(**p_t).to('cpu').numpy()
    p_embs.append(p_emb)
    tokenized_corpus.append(tokenize_fn(p, padding="max_length", truncation=True, return_tensors='pt'))
p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)


100%|██████████| 55963/55963 [18:20<00:00, 50.85it/s]


In [14]:
with torch.no_grad():
  q_encoder.eval()
  q_seqs_val = tokenizer(query, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
  q_emb = q_encoder(**q_seqs_val).to('cpu')  #(num_query, emb_dim)

p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)

print(p_embs.size(), q_emb.size())

torch.Size([55963, 768]) torch.Size([240, 768])


In [27]:
k=10

In [15]:
def get_top_n(bm25, query, documents, n=10):
    assert bm25.corpus_size == len(documents), "The documents given don't match the index corpus!"

    scores = bm25.get_scores(query)

    top_n_idx = np.argsort(scores)[::-1][:n]
    doc_score = scores[top_n_idx]
        
    return doc_score, top_n_idx

In [35]:
from retrieval import MyBm25

In [36]:
def get_sparse_embedding(bm25,contexts, tokenize_fn,data_path ="../",k1=1.5, b=0.75, epsilon=0.25) -> NoReturn:

    """
    Summary:
        Passage Embedding을 만들고
        TFIDF와 Embedding을 pickle로 저장합니다.
        만약 미리 저장된 파일이 있으면 저장된 pickle을 불러옵니다.
    """
    bm25_name = f"bm25.bin"
    bm25_path = os.path.join(data_path, bm25_name)
    if os.path.isfile(bm25_path):
        with open(bm25_path, "rb") as file:
            bm25 = pickle.load(file)
        print("Embedding bm25 pickle load.")
    else:
        print("Building bm25... It may take 1 minute and 30 seconds...")
        # bm25 must tokenizer first 
        # because it runs pool inside and this cuases unexpected result.
        tokenized_corpus = []
        for c in contexts:
            tokenized_corpus.append(tokenize_fn(c))
        bm25 = MyBm25(tokenized_corpus, k1 = k1, b = b, epsilon=epsilon)
        with open(bm25_path, "wb") as file:
            pickle.dump(bm25, file)
        print("bm25 pickle saved.")
    return bm25

In [37]:
bm25 = MyBm25(tokenized_corpus)

In [47]:
bm25

In [41]:
se = get_sparse_embedding(bm25,contexts,tokenize_fn)

Embedding bm25 pickle load.


In [46]:
bm25 = se

In [61]:
len(bm25.get_scores(query[0])

55963

In [62]:
doc_scores = []
for q in tqdm(query):
    tok_q = tokenize_fn(q)
    scores = bm25.get_scores(tok_q)
    doc_scores.append(scores)
print("done!")
#return doc_scores, doc_indices

100%|██████████| 240/240 [02:40<00:00,  1.50it/s]

done!


In [66]:
doc_scores = torch.tensor(np.array(doc_scores))

torch.Size([240, 55963]) 240 55963


In [127]:
doc_scores = torch.tensor(np.array(doc_scores))
dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
print(dot_prod_scores.size(),len(doc_scores),len(doc_scores[0]))

dpr_score = softmax(dot_prod_scores,dim=1)
bm25_score = softmax(doc_scores,dim=1)


total_score = []
for idx in range(len(query)):
    total_score.append((dpr_score[idx]*0.1+bm25_score[idx]).tolist())
total_score = torch.tensor(np.array(total_score))


In [163]:
doc_scores = torch.tensor(np.array(doc_scores))
dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
print(dot_prod_scores.size(),len(doc_scores),len(doc_scores[0]))

dpr_score = softmax(dot_prod_scores,dim=1)
bm25_score = softmax(doc_scores,dim=1)


total_score = []
for idx in range(len(query)):
    total_score.append((dpr_score[idx]*0.1+bm25_score[idx]).tolist())
total_score = torch.tensor(np.array(total_score))


k_list = [1,3,5,10,15,20]
rate_list=[0.0,0.1,0.2,0.5,1.0,1.1,1.2]

score_Dict = {k:[] for k in k_list}


for k in tqdm(k_list):
    for rate in rate_list:
        total_score = []
        for idx in range(len(query)):
            total_score.append((dpr_score[idx]*rate+bm25_score[idx]).tolist())
        total_score = torch.tensor(np.array(total_score))

        ranks = torch.argsort(total_score, dim=1, descending=True).squeeze()
        context_list = []

        for index in range(len(ranks)):
            k_list = []
            for i in range(k):
                k_list.append(contexts[ranks[index][i]])
            context_list.append(k_list)

        correct= 0
        correct_list = []
        for index in range(len(context_list)):
            for ctx_idx in range(len(context_list[0])) :
                if ground_truth[index] == context_list[index][ctx_idx]:
                    correct+=1 
                    correct_list.append(ctx_idx+1)
        score_Dict[k].append([(correct/len(context_list)),(sum(correct_list)/len(correct_list))])

torch.Size([240, 55963]) 240 55963


100%|██████████| 6/6 [01:53<00:00, 18.91s/it]


In [164]:
score_Dict

{1: [[0.6583333333333333, 1.0],
  [0.6541666666666667, 1.0],
  [0.6541666666666667, 1.0],
  [0.6583333333333333, 1.0],
  [0.6625, 1.0],
  [0.6416666666666667, 1.0],
  [0.6333333333333333, 1.0]],
 3: [[0.7916666666666666, 1.194736842105263],
  [0.8, 1.25],
  [0.7958333333333333, 1.2356020942408377],
  [0.7875, 1.2275132275132274],
  [0.7791666666666667, 1.2299465240641712],
  [0.775, 1.2365591397849462],
  [0.7708333333333334, 1.2378378378378379]],
 5: [[0.85, 1.4166666666666667],
  [0.8375, 1.3880597014925373],
  [0.8375, 1.3930348258706469],
  [0.8333333333333334, 1.405],
  [0.8166666666666667, 1.3928571428571428],
  [0.8166666666666667, 1.413265306122449],
  [0.8125, 1.4102564102564104]],
 10: [[0.9083333333333333, 1.8073394495412844],
  [0.9083333333333333, 1.834862385321101],
  [0.9125, 1.9223744292237444],
  [0.9, 1.8657407407407407],
  [0.8916666666666667, 1.9205607476635513],
  [0.8916666666666667, 1.9532710280373833],
  [0.8916666666666667, 1.9672897196261683]],
 15: [[0.920833

In [139]:
k=10
rate=0.2
total_score = []
for idx in range(len(query)):
    total_score.append((dpr_score[idx]*rate+bm25_score[idx]).tolist())
total_score = torch.tensor(np.array(total_score))

ranks = torch.argsort(total_score, dim=1, descending=True).squeeze()
context_list = []

for index in range(len(ranks)):
    k_list = []
    for i in range(k):
        k_list.append(contexts[ranks[index][i]])
    context_list.append(k_list)
    

In [150]:
sum([1,2,3])/3

2.0

In [161]:
correct= 0
correct_list = []
for index in range(len(context_list)):
    for ctx_idx in range(len(context_list[0])) :
        if ground_truth[index] == context_list[index][ctx_idx]:
            correct+=1 
            correct_list.append(ctx_idx+1)
score_Dict[k].append([(correct/len(context_list)),(sum(correct_list)/len(correct_list))])

In [158]:
score_Dict[10] = []

In [162]:
score_Dict[10]

[[0.9125, 1.9223744292237444]]

In [144]:
context_list[1]

['요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & 니피싱 철도의 설립 인가가 떨어졌을 때였다. 스카버러 남쪽과 코보콘크 마을을 잇는 철도 노선 공사가 시작되었고 1871년 6월에 억스브릿지까지 철도가 완공되었다. 이 노선은 이후 1871년 11월에 캐닝턴까지 연장되었고 1872년 11월에 코보콘크까지 완공되었다. 이 노선의 철로 궤간은 1067mm로 협궤 노선이였다. 목재와 장작 수요가 선로 용량이 넘칠 정도로 많았지만 1870년대에 다른 철도와 마찬가지로 경제적 난황을 이기 못해 수익이 줄어들었다. 투자자들은 수익이 줄어든 이유로 철도가 협궤로 지어져서 다른 표준궤 노선과 연계하여 화물 운송을 할 수 없는 점을 들었다 1881년 7월, 미들랜드 철도가 토론토 & 니피싱 철도를 인수하였고 이 노선은 표준궤로 전환되었다. 미들랜드 철도의 영향은 스카버러에서도 볼 수 있는데 노선 동쪽을 따라 나란히 달리는 미들랜드 애비뉴가 이 철도 회사의 이름을 따서 지어졌다. 1884년, 미들랜드 철도는 그랜드 트렁크 철도에 인수되었고 이는 이후 캐나다 내셔널 철도에 다시 인수되었다. CN은 이 노선을 억스브릿지 선으로 명명하였다 억스브릿지 선을 따라 달리는 여객 철도는 통근객을 위한 열차가 아니였다. 처음에는 론빌에 있는 미들랜드정션역과 토론토를 잇는 열차가 두 대씩 양방향으로 운행했고 한 대는 북동쪽으로 코보콘크까지 운행하였다. 토론토 북동쪽에 있던 시골 마을의 수요는 저조하기만 하였으며 20세기에 들어서서 도로 개량과 자동차 보편화로 쇠락의 길을 걷게 되었다 1955년에는 코보콘크에서 억스브릿지까지 여객 열차 운행이 중단되었으며 30년 뒤 선로 또한 폐선되었다. 1963년에는 억스브릿지까지 이어지는 여객열차 운행이 중단되고 CN 열차는 토론토 유니언역에서 마컴으로 가는 열차를 5시 20분에 딱 한 대 운행하였다. 토론토로 돌아오는 열차는 존재하지 않았다',
 '이 역이 교통 허브로 거듭나게 된 것은 거의 우연에 가까웠으며 기존에 스카버러

In [ ]:
"""
학습 : train -> 검증 : valid question을 전체 wiki에서 뽑기 
DPR은 0.2쯤둬야 약간 오르고 나머지는 오히려 낮추었음

"""